In [5]:
import jax.numpy as jnp
import jax

### 1.2 Spatial Vectors

Normally the equations of motion for a rigid body are:

$$\begin{equation} \tag{1.4} f = ma \quad \text{and} \quad n = I \dot{\omega} + \omega \times I \omega \end{equation}$$

The first says force (through the center of mass) relates to linear acceleration about the center of mass. The second says torque relates to angular acceleration.

Using spatial vectors, we can write

$$\begin{equation} \tag{1.5} f = Ia + v \times^* I v \end{equation}$$

where $\times^*$ is the spatia vector cross product.

### 2.1 Mathematical Preliminaries

* $R^n$ coordinate vectors
* $E^n$ euclidean vectors (coordinate vectors with inner product defined)
* $M^n$ spatial motion vectors (velocity acceleration)
* $F^n$ spatial force vectors (force, impulse, momentum)

**Dual of a vector Space:** If the dual of $V$ is $V^*$, then there's a scalar product between elements in each $v \cdot u$

**Dual Coordinates:** When $u$ and $v$ are expressed in dual coordinates, then $u \cdot v = u^Tv$

**Dyad:** Expressions like $a b \cdot$ are called dyads. Think of it as a linear operator that maps vectors to vectors. Applied to $c$, this is $a(b \cdot c)$. Note that $ab\cdot  = ab^T$.

### 2.2 Spatial Velocity

$$\begin{equation} \tag{2.1} v_P = v_O + \omega \times \overset{\rightarrow}{OP} \end{equation}$$

computes the velocity of point $P$ given the linear and angular velocity of a point $O$, and the position of $P$ relative to $O$.

$$\begin{equation} \tag{2.2} \mathcal{D}_O = \{d_{Ox}, d_{Oy}, d_{Oz}, d_x, d_y, d_z\} \subset M^6 \end{equation}$$

is the Plücker basis defining a coordinate system on the space of motion vectors.

### 2.3 Spatial Force

$$\begin{equation} \tag{2.6} n_P = n_O + f \times \overset{\rightarrow}{OP} \end{equation}$$

computes the total moment about a point $P$ given the moment about $O$ and a force through $O$.

$$\begin{equation} \tag{2.7} \mathcal{E}_O = \{e_x, e_y, e_z, e_{Ox}, e_{Oy}, e_{Oz}\} \subset F^6 \end{equation}$$

is the Plücker basis.

### 2.5 Line Vectors and Free Vectors

* **Line vector:** 5 numbers
* **Free vector:** 3 numbers

$M^6$ and $F^6$ are twists and wrenches respectively.

### 2.6 Scalar Product

If $X$ transforms $m \in M^6$ then the equivalent transform on $f \in F^6$ is $X^{-T}$. This is because

$$(Xm)^T \cdot (X^{-T}f) = m^T X^T X^{-T}f = m^T f$$

### 2.7 Using Spatial Vectors

* $M^6$ is for velocity, acceleration, infinitesimal displacement
* $F^6$ is for force, impulse, momentum

* **Power:** force time velocity $f \cdot v$
* **Momentum:** Intertia times velocity $Iv$

#### Kinematic Chains

Given two adjecent bodies in the kinematic chain with velocities $v_i$ and $v_{i-1}$, the relative velocity is $v_{Ji} = v_i - v_{i-1}$. If the joint between the bodies has a single degree of freedom, then we can express the relative velocity as

$$\begin{equation} \tag{2.15} v_{Ji} = s_i \dot{q}_i \end{equation}$$

where $s_i$ is the "axis vector."}}

### 2.8 Coordinate Transforms

$^BX_A$ transforms from $A$ to $B$ coordinates.

#### Rotation

If we have spatial motion vectors ${^B}\hat{m}, {^A}\hat{m} \in M^6$ with a common origin $O$, then we can express the transform between the two coordinate systems with a pure rotation


$$\begin{equation} \tag{2.19} {^B}\hat{m}  = \begin{bmatrix} {^B}E_{A} & 0 \\ 0 & {^B}E_{A} \end{bmatrix} {^A}\hat{m} \end{equation}$$

Where $E$ is the rotation matrix. And note that the transform for spatial force vectors is equivalent in this case ${^B}X_A = {^B}X_A^{-T}$

#### Translation

Given two points in space $O$ and $P$ with two frames having the same orientation. The transform between spatial motion at the two frames is

$$\begin{equation} \tag{2.21} {^B}\hat{m} = \begin{bmatrix} 1 & 0 \\ -r \times & 1 \end{bmatrix} {^A}\hat{m} \end{equation}$$

where $r = \overset{\rightarrow}{OP}$ and $r \times$ is a 3x3 matrix for the cross product (skew). Note that the equivalent transform for spatial force vectors is

$$\begin{equation} \tag{2.22} {^P}X_O^{-T} = \begin{bmatrix} 1 & -r \times \\ 0 & 1 \end{bmatrix} {^A}X_B \end{equation}$$

In [6]:
def SO3_hat(v):
    return jnp.array([[0, -v[2], v[1]],
                      [v[2], 0, -v[0]],
                      [-v[1], v[0], 0]])

# Test the skew function
seed = 0
key = jax.random.PRNGKey(seed)
for _ in range(10):
    u = jax.random.normal(key, (3,))
    v = jax.random.normal(key, (3,))
    if (jnp.linalg.norm(SO3_hat(u) @ v - jnp.cross(u, v))) > 1e-6:
        print("FAIL")

#### General Transforms

The motion vector transform is

$$\begin{equation} \tag{2.24} {^B}X_A =
\begin{bmatrix} E & 0 \\ 0 & E \end{bmatrix}
\begin{bmatrix} 1 & 0 \\ -r\times & 1 \end{bmatrix}
=\begin{bmatrix} E & 0 \\ -Er\times & E \end{bmatrix}\end{equation}$$

And the force vector transform is

$$\begin{equation} \tag{2.25} {^B}X_A^{-T} =
\begin{bmatrix} E & 0 \\ 0 & E \end{bmatrix}
\begin{bmatrix} 1 & -r\times \\ 0 & 1 \end{bmatrix}
=\begin{bmatrix} E & -Er\times \\ 0 & E \end{bmatrix}\end{equation}$$

In [7]:
def make_motion_transform(R, t):
    assert R.shape == (3, 3)
    assert t.shape == (3,)
    return jnp.block([[R, jnp.zeros((3, 3))], [-R @ SO3_hat(t), R]])

def make_force_transform(R, t):
    assert R.shape == (3, 3)
    assert t.shape == (3,)
    return jnp.block([[R, -R @ SO3_hat(t)], [jnp.zeros((3, 3)), R]])

make_motion_transform(jnp.eye(3), jnp.array([2, 0, 0]))

DeviceArray([[ 1.,  0.,  0.,  0.,  0.,  0.],
             [ 0.,  1.,  0.,  0.,  0.,  0.],
             [ 0.,  0.,  1.,  0.,  0.,  0.],
             [ 0.,  0.,  0.,  1.,  0.,  0.],
             [ 0.,  0.,  2.,  0.,  1.,  0.],
             [ 0., -2.,  0.,  0.,  0.,  1.]], dtype=float32)

### 2.9 Spatial Cross Products

If $r$ is some vector rotating with angular velocity $\omega$, then the derivative is $\dot{r} = \omega \times r$. This idea extends to spatial vectors $m, v \in M^6$ and $f \in F^6$.

$$\begin{equation} \tag{2.29} \dot{m} = v \times m \end{equation}$$

and 

$$\begin{equation} \tag{2.30} \dot{f} = v \times^* f \end{equation}$$

where $\times^*$ is the dual of $\times$ which has the property $v \times^* = -v \times^T$.

In [8]:
class SpatialMotionVector(jnp.ndarray):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        assert self.shape == (6,)

    def SO3_hat(self):
        w, v = self[:3], self[3:]
        return jnp.block([[SO3_hat(w), 0],
                          [SO3_hat(v), SO3_hat(w)]])

    def cross(self, other):
        if isinstance(other, SpatialMotionVector):
            return SpatialMotionVector(self.skew() @ other)
        elif isinstance(other, SpatialForceVector):
            return SpatialForceVector(-self.skew().T @ other)
        else:
            return super().cross(other)


class SpatialForceVector(jnp.ndarray):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        assert self.shape == (6,)

### 2.10 Differentiation

#### Differentiation in Moving Coordinates

Suppose we have a frame $A$ moving with velocity $v_A$. Given spatial motion vector ${^A}m \in M^6$, we can compute the time derivative

$$\begin{equation} \tag{2.36} {^A} \left( \frac{d m}{d t} \right)  = \frac{d {^A}m}{dt} + {^A}v_A \times {^A}m \end{equation}$$

where ${^A}v_A$ is $v_A$ in $A$ coordinates.

#### Dot and Ring Notation

We need to differentiate between the derivate expressed in $A$ coordinates, and differentiating a spatial vector in $A$ coordinates.

$$\begin{equation} \tag{2.39} {^A}\dot{v} = {^A} \left( \frac{d m}{d t} \right)  \end{equation}$$

$$\begin{equation} \tag{2.40} {^A}\mathring{v} =   \frac{d {^A} m}{d t}  \end{equation}$$

Given two frames having velocity $v_A$ and $v_B$ respectively, we can compute the time derivative of the transform between the two frames as

$$\begin{equation} \tag{2.45} \frac{d}{dt}{^B}X_A = {^B}(v_A - v_B) \times {^B}X_A \end{equation}$$

### 2.12 Momentum

If a body's center of mass coincides with fixed point $C$, and has a spatial velocity $\omega$ and $v_C$ with rotational inertia $I_C$ about its center of mass, then the linear momentum is $h = mv_C$ and the intrinsic angular momentum is $h_C = I_C \omega$.

The moment of momentum about a point $O$ is $h_O = h_C + \overset{\rightarrow}{OC} \times h$.

Spatial momentum vectors are elements of $F^6$.

### 2.13 Inertia

Spatial inertia relates velocity to momentum; it's a mapping from $M^6$ to $F^6$. It's a 6x6 matrix with the following form

$$\begin{equation} \tag{2.62} I = \begin{bmatrix} I_C & 0 \\ 0 & m1 \end{bmatrix} \end{equation}$$

* Spatial inertia is a symmetric dyadic tensor.
* $\Phi = I^{-1}$ is the inverse inertia.
* Inertias sum
* The kinetic energy of a body is $T = \frac{1}{2} v \cdot Iv$


### 3.1 Equations of Motion

$$\begin{equation} \tag{3.1} H(q) \ddot{q} + C(q, \dot{q}) = \tau \end{equation}$$

Where $H(q)$ is the generalized inertia matrix, $C$ is the generalized bias force vector, and $\tau$ is the generalized force vector.

### 3.2 Constructing Equations of Motion

If there is a motion constraint, we can model this with a constaint force

$$\begin{equation} \tag{3.1} H(q) \ddot{q} + C(q, \dot{q}) = \tau + \tau_c \end{equation}$$

where $\tau_c$ delivers zero power: $\tau_c \cdot \dot{q} = 0 \quad \forall \dot{q}$.

#### Applying Implicit Motion Constraints

If we have implicit motion constraints

* $\phi(q)  = 0$
* $K\dot{q} = 0$
* $K\ddot{q} = k$

then $\tau_c = K^T \lambda$ where $\lambda$ can be thought of as the Lagrange multipliers. The equation of motion becomes

$$\begin{equation} \tag{3.17}
\begin{bmatrix} H & K^T \\ K & 0 \end{bmatrix}
\begin{bmatrix} \ddot{q} \\ -\lambda \end{bmatrix}
=\begin{bmatrix} \tau - C \\ k \end{bmatrix}
\end{equation}$$

which is $n + n_c$ equations where $n$ is the number of dofs and $n_c$ is the number of constraints.

#### Applying Explicit Motion Constraints

If we have explicit motion constraints

* $q = \gamma(y)$
* $\dot{q} = G \dot{y}$
* $\ddot{q} = G \ddot{y} + g$

then we can project the equations of motion onto the constraint space by

* $H_G = G^T H G$
* $C_G = G^T(C  + H g)$
* $u = G^T \tau$

to get

$$\begin{equation} \tag{3.21} H_g \ddot{y} + C_G = u \end{equation}$$



### 3.4 Classification of Constriants

* _holonomic_ constraints constrain the positions. $\phi(q) = 0$
* _nonholonomic_ constraints constrain the positions and velocities $\phi(q, \dot{q}) = 0$

A system with nonholonomic constraints has more positional degrees of freedom than velocity degrees of freedom.

### 3.5 Joint Constraints

Joints constraint the relative velocity between two bodies $v_J = v_s - v_p$ where
the subscripts denote the successor and predecessor bodies. If the joint does not depend on time:

$$\begin{equation} \tag{3.33} v_J = S(q) \dot{q} \end{equation}$$

where $S$ is a $6 \times n$ matrix is the joint motion subspace.

Let $f_J$ be the force transmitted from the predecessor to the successor, so $-f_J$ is the force acting on the predecessor. The joint force can be split into the _active_ and _constraint_ force components

$$\begin{equation} \tag{3.34} f_J = T_a \tau + T \lambda \end{equation}$$

where $T_a$ is the active force subspace and $T$ is the constraint force subspace, satisfying 

* $T_a^T S = 1$
* $T^T S = 0$

The joint acceleration constaints are (recalling $v_s$ is the spatial velocity of the successor)

$$\begin{equation} \tag{3.40} a_J = S\ddot{q} + \dot{S} \dot{q} = S \ddot{q} + v_s \times v_J \end{equation}$$


### Dynamics of a Constrained Rigid Body

$$f + f_c + f_g = Ia + v\times^* Iv$$

* $f$ is the active force
* $f_g$ is the gravitational force
* $f_c$ is the constraint force

We want to find by $a$ and $f_c$.

#### Method 1 (Direct)

Recall

* $v = S \dot{q}$ maps the generalized velocity to the spatial velocity
* $S^T f_c = 0$ says that the constaint force is orthogonal to the active force subspace
* $a = S \ddot{q} + \dot{S}\dot{q}$ is the acceleration constraint

Combining those we can get

$$\begin{equation} \tag{3.51} \ddot{q} = (S^T I S)^{-1} S^T (f - I\dot{S}\dot{q} - p) \end{equation}$$

where $p = v \times^* Iv - f_g$ is the bias force (from gravity and such). We convert from generalized velocities $\ddot{q}$ to spatial velocities $a$ by

* $a = \Phi f + b$ where
* $\Phi = (S^T I S)^{-1} S^T$ is the apparent inverse inertia
* $b = \dot{S}\dot{q} - \Phi (I \dot{S}\dot{q} + p)$ is the bias acceleration

#### Method 2 (Lagrange multipliers)

#### Method 3 (Project into generalized coordinates)

The projection throws away information, and allows dynamics simulation, but not disentangling the constraint forces from the other forces.


### 3.7 Dynamics of a Multibody System

$$\begin{equation} \tag{3.66} f = Ia + p \end{equation}$$

where $f$ is the total force, $I$ is the total inertia, $a$ is the total acceleration, and $p$ is the total bias force. If there are $N_B$ bodies, then $f, a, p$ are all $6N_B$ vectors. The inertia matrix is $6N_B \times 6N_B$ block diagonal.

#### Joint Motion from Body Motion

Joint constraints are defined in terms of velocity, so we need a way to go from body velocity to joint velocity. The velocity (and acceleration) of joint $i$ is the difference between the bodies

$$ v_{Ji} = v_{succ(i)} - v_{pred(i)} $$

We can define a matrix $P$ of blocks of $-1, 0, 1$ to map from body velocity to joint velocity: $v_J = P^T v$ where $P$ is $6N_B \times 6N_J$. Note that this matrix can also be used to map from joint forces to body forces: $f = P f_J$.

#### Motion Constaints

For a joint $i$ with motion subspace $S_i$ and complement constraint subspace $T_i$, the motion constraint is $T_i^T v_{Ji} = 0$ and the acceleration constraint is $T_i^T a_{Ji}  + \dot{T_i}^T v_{Ji} = 0$.

As before we can gather all the motion constraints into a block-diagonal matrix $T$ of shape $N_J \times N_C$ where $N_C$ is the number of constraints. With $P$, we can express constrints in terms of body velocity and acceleration:

$$\begin{equation} \tag{3.71} T^T P^T a + \dot{T}^T P^T v = 0 \end{equation}$$

#### Constraint Forces

The force at joint $i$ is the sum of the active and constraint forces: $f_{Ji} = T_{ai} \tau_i + T_i \lambda_i$ where $T_{ai}$ spans the active force subspace. We can collect this into a big block-diagonal equation as before.

#### Final equation

By combining the above we get

$$\begin{equation} \tag{3.74}
\begin{bmatrix} I & PT \\ T^TP^T & 0 \end{bmatrix}
\begin{bmatrix} a \\ -\lambda \end{bmatrix} =
\begin{bmatrix} P T_a \tau - p \\ -\dot{T}^T P^T v \end{bmatrix} 
\end{equation}$$



### 5.3 The Recursive Newton-Euler Algorithm

 1. calculate velocities and accelerations of bodies
 2. calculate forces on bodies
 3. calculate joint forces

 #### Step 1

 $$\begin{equation} \tag{5.7} v_i = v_{\lambda(i)} + S_i \dot{q}_i \end{equation}$$

 and for acceleration

 $$\begin{equation} \tag{5.8} a_i = a_{\lambda(i)} + S_i \ddot{q}_i + \dot{S}_i \dot{q}_i \end{equation}$$

 #### Step 2

$$\begin{equation} \tag{5.9} f_i = I_i a_i + v_i \times^* I_i v_i \end{equation}$$

### Notation

 * $\bm{\lambda}_j$ is the $n_{cj}$-dimensional vector of constraint-force variables
 * $\lambda$ is the parent array where $\lambda(i)$ is the parent of body $i$
 * $\mu(i)$ is the set of children of body $i$ 
 * $\bm{S}_i$ motion subspace of joint $i$
 * $\bm{I}_i$ is the inertia of body $i$

 